# Retrieving XMLs from eSc
We retrieve xmls of document parts from eScriptorium,
in order to prepare them for processing with Passim.

## 0 - Notebook initialization

In [2]:
from functions import *
from packages import *

switching to  msIA


# 1 - get document parts xmls from eScriptorium

In [16]:
doc_pk = 3585
get_basic_info(doc_pk)


get document segmentation ontology for document:  3585
https://msia.escriptorium.fr/api/documents/3585/
Document: 3585  with  901  parts
region types: [{'pk': 5399, 'name': 'Catchword'}, {'pk': 5402, 'name': 'FooterCentral'}, {'pk': 5718, 'name': 'Handwritten'}, {'pk': 5401, 'name': 'Header'}, {'pk': 4, 'name': 'Illustration'}, {'pk': 5403, 'name': 'MainCentral'}, {'pk': 5400, 'name': 'Margin'}, {'pk': 5719, 'name': 'Stamp'}]
line types: [{'pk': 14006, 'name': 'Handwritten'}, {'pk': 13854, 'name': 'MainCentral'}, {'pk': 13868, 'name': 'NotMain'}]
transcription_level_list: [{'pk': 8954, 'name': 'kraken:sinai_no_voc_61', 'archived': False, 'avg_confidence': 0.9812029473975379}, {'pk': 8956, 'name': 'kraken:HebScriptsHumaChar02_6', 'archived': False, 'avg_confidence': 0.9829252948818921}, {'pk': 8590, 'name': 'manual', 'archived': False, 'avg_confidence': None}]


(901,
 [{'pk': 8954,
   'name': 'kraken:sinai_no_voc_61',
   'archived': False,
   'avg_confidence': 0.9812029473975379},
  {'pk': 8956,
   'name': 'kraken:HebScriptsHumaChar02_6',
   'archived': False,
   'avg_confidence': 0.9829252948818921},
  {'pk': 8590, 'name': 'manual', 'archived': False, 'avg_confidence': None}],
 [{'pk': 5399, 'name': 'Catchword'},
  {'pk': 5402, 'name': 'FooterCentral'},
  {'pk': 5718, 'name': 'Handwritten'},
  {'pk': 5401, 'name': 'Header'},
  {'pk': 4, 'name': 'Illustration'},
  {'pk': 5403, 'name': 'MainCentral'},
  {'pk': 5400, 'name': 'Margin'},
  {'pk': 5719, 'name': 'Stamp'}],
 [{'pk': 14006, 'name': 'Handwritten'},
  {'pk': 13854, 'name': 'MainCentral'},
  {'pk': 13868, 'name': 'NotMain'}])

In [17]:
# get the region type list
region_type_list, line_type_list = get_document_segmentation_ontology(doc_pk,print_status=False)
print(region_type_list)

# get the region type pk list
region_type_pk_list = [region_type['pk'] for region_type in region_type_list]
print(region_type_pk_list)

[{'pk': 5399, 'name': 'Catchword'}, {'pk': 5402, 'name': 'FooterCentral'}, {'pk': 5718, 'name': 'Handwritten'}, {'pk': 5401, 'name': 'Header'}, {'pk': 4, 'name': 'Illustration'}, {'pk': 5403, 'name': 'MainCentral'}, {'pk': 5400, 'name': 'Margin'}, {'pk': 5719, 'name': 'Stamp'}]
[5399, 5402, 5718, 5401, 4, 5403, 5400, 5719]


In [18]:
# get all the parts from the document
all_parts = get_all_parts(doc_pk)

# get the part pk list
part_pk_list = [ part['pk'] for part in all_parts ]
print(part_pk_list)


[611819, 611822, 611825, 611828, 611831, 611835, 611838, 611841, 611843, 611846, 611849, 611852, 611855, 611859, 611862, 611865, 611868, 611871, 611873, 611876, 611878, 611881, 611884, 611886, 611889, 611892, 611895, 611898, 611900, 611903, 611906, 611909, 611912, 611915, 611917, 611919, 611921, 611924, 611927, 611930, 611932, 611935, 611938, 611941, 611944, 611947, 611951, 611954, 611957, 611960, 611963, 611966, 611970, 611973, 611975, 611978, 611981, 611984, 611987, 611990, 611994, 611998, 612001, 612005, 612008, 612011, 612015, 612018, 612021, 612025, 612028, 612030, 612033, 612037, 612040, 612043, 612047, 612050, 612054, 612058, 612062, 612066, 612069, 612072, 612075, 612079, 612082, 612085, 612088, 612091, 612093, 612096, 612099, 612102, 612105, 612108, 612111, 612113, 612116, 612119, 612121, 612124, 612127, 612130, 612133, 612136, 612139, 612142, 612145, 612147, 612149, 612152, 612154, 612157, 612158, 612159, 612160, 612161, 612162, 612163, 612164, 612165, 612166, 612167, 612168,

In [19]:
# part_pk_list = [612540, 612541, 612543]

# Choose the transcription level
tr_level_pk = 8954

# get the xmls of the parts from eScriptorium
export_xml(doc_pk,part_pk_list,tr_level_pk,region_type_pk_list,include_undefined = False, include_orphan = False, file_format = 'alto',include_images = False, print_status = True)


200
b'{"status":"ok"}'


<Response [200]>

--------
